# Morph1x - Object Detection Demo

This notebook demonstrates the Morph1x video tracking system for detecting living beings in images and videos.

## Features
- 🎯 Real-time object detection using YOLOv8
- 🟢 Green bounding boxes around living beings
- 🏷️ Labels showing object names and confidence scores
- 📊 Detection statistics and visualization


In [ ]:
# Import required libraries
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from src.detection import create_detector
from src.utils import draw_detections, draw_info_panel, FPSMeter
from src.config import PRIMARY_LIVING_BEINGS

print("✅ Libraries imported successfully!")
print(f"🎯 Detecting: {list(PRIMARY_LIVING_BEINGS.values())}")


## 1. Initialize the Detector

Let's create our YOLOv8-based detector for living beings.


In [ ]:
# Initialize the detector
detector = create_detector()

print("🎯 YOLOv8 detector initialized!")
print("📁 Model loaded from:", detector.model_path)


## 2. Test Detection on Sample Image

Let's create a test image and run detection on it.


In [ ]:
# Create a test image (you can replace this with your own image)
test_image = np.zeros((480, 640, 3), dtype=np.uint8)

# Draw some shapes to simulate objects
cv2.rectangle(test_image, (100, 100), (300, 400), (100, 100, 100), -1)  # Gray rectangle
cv2.circle(test_image, (450, 200), 80, (150, 150, 150), -1)  # Gray circle
cv2.putText(test_image, "Test Image", (250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Run detection
detections = detector.detect_living_beings(test_image)

print(f"🔍 Found {len(detections)} detections")
for i, det in enumerate(detections):
    print(f"  {i+1}. {det['class_name']} (confidence: {det['confidence']:.2f})")

# Display the image
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
plt.title("Test Image")
plt.axis('off')
plt.show()


## 3. Load and Process Your Own Image

Replace the path below with your own image to test detection.


In [ ]:
# Load your own image (uncomment and modify the path)
# image_path = "path/to/your/image.jpg"
# if os.path.exists(image_path):
#     image = cv2.imread(image_path)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     
#     # Run detection
#     detections = detector.detect_living_beings(image)
#     
#     # Draw detections
#     result_image = draw_detections(image.copy(), detections)
#     result_image = draw_info_panel(result_image, detection_count=len(detections))
#     result_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
#     
#     # Display results
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
#     ax1.imshow(image_rgb)
#     ax1.set_title("Original Image")
#     ax1.axis('off')
#     
#     ax2.imshow(result_rgb)
#     ax2.set_title(f"Detections: {len(detections)}")
#     ax2.axis('off')
#     
#     plt.tight_layout()
#     plt.show()
#     
#     print(f"🎯 Found {len(detections)} living beings:")
#     for det in detections:
#         print(f"  - {det['class_name']}: {det['confidence']:.2f}")
# else:
#     print("📁 Please update the image_path variable with a valid image file")

print("💡 To test with your own image, uncomment and modify the code above!")


## 4. Video Processing Demo

Let's process a video file frame by frame to see the detection in action.


In [ ]:
# Video processing demo
def process_video_demo(video_path, max_frames=10):
    """Process a video and show detection results."""
    if not os.path.exists(video_path):
        print(f"❌ Video file not found: {video_path}")
        return
    
    cap = cv2.VideoCapture(video_path)
    fps_meter = FPSMeter()
    frame_count = 0
    
    print(f"🎬 Processing video: {video_path}")
    print("📊 Showing first few frames with detections...")
    
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run detection
        detections = detector.detect_living_beings(frame)
        
        # Draw detections
        result_frame = draw_detections(frame.copy(), detections)
        result_frame = draw_info_panel(
            result_frame, 
            fps=fps_meter.get_fps(),
            detection_count=len(detections)
        )
        
        # Convert to RGB for matplotlib
        result_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)
        
        # Display frame
        plt.figure(figsize=(12, 8))
        plt.imshow(result_rgb)
        plt.title(f"Frame {frame_count + 1} - Detections: {len(detections)}")
        plt.axis('off')
        plt.show()
        
        print(f"Frame {frame_count + 1}: {len(detections)} detections")
        for det in detections:
            print(f"  - {det['class_name']}: {det['confidence']:.2f}")
        
        fps_meter.update()
        frame_count += 1
    
    cap.release()
    print(f"✅ Processed {frame_count} frames")

# Uncomment to process your video
# process_video_demo("path/to/your/video.mp4", max_frames=5)

print("💡 To process your video, uncomment the line above and update the path!")


## 5. Detection Statistics

Let's analyze the detection capabilities and performance.


In [ ]:
# Display detection capabilities
print("🎯 Morph1x Detection Capabilities")
print("=" * 40)
print("Living beings that can be detected:")
for class_id, class_name in PRIMARY_LIVING_BEINGS.items():
    print(f"  {class_id:2d}. {class_name}")

print(f"\n📊 Total classes: {len(PRIMARY_LIVING_BEINGS)}")
print(f"🎯 Primary focus: Humans and animals")

# Performance test
import time
test_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)

start_time = time.time()
detections = detector.detect_living_beings(test_image)
end_time = time.time()

print(f"\n⚡ Performance Test:")
print(f"  Processing time: {(end_time - start_time)*1000:.2f} ms")
print(f"  Detections found: {len(detections)}")

print(f"\n✅ Morph1x is ready for real-time detection!")
